## Descrição do Notebook

Este notebook realiza uma série de operações de processamento para o tratamento e a inserção de novos dados, nos dados já tratados referentes as denúncias de violência contra a mulher, fornecido pelo Ministério dos Direitos Humanos e da Cidadania. As funções definidas e os passos executados incluem leitura de dados, padronização de formatos, limpeza e classificação baseada em critérios específicos. O objetivo final é consolidar informações de várias tabelas para análises relacionadas a denúncias registradas através do canal Ligue 180.

### Funções Definidas

- `RemoveAcentosCaixaAlta(celula)`: Converte strings para letras maiúsculas e remove acentos.
- `ClassificaProfissao(profissao, arquivoComProfissoes)`: Classifica profissões em categorias (definidadas no dicionário de dados do ENEM) baseadas nos radicais das palavras.
- `PadronizaMunicipio(municipio)`: Extrai o nome do município de strings contendo múltiplas informações.
- `PadronizaPais(pais)`: Similar à função de município, mas aplicada ao nome do país.

### Processo de Leitura dos Dados

O notebook lê dados de múltiplos arquivos CSV e Excel, que incluem: a nova tabela contendo mais dados, as tabelas com os dados já tratados anteriormente no projeto de pesquisa, ums lista de municípios usada no tratamento, a tabela de radicais de profissões usadas para categoriza-las conforme dicionário de dados do ENEM.

### Verificação e Atualização de Padrões

Verifica se os novos dados seguem o mesmo padrão das tabelas existentes, utilizando a tabela "Comparação dos campos de cada tabela.xlsx" de referência. Se discrepâncias são encontradas, as diferenças são indicadas para adequação manual. Se não houverem discrepâncias, o arquivo de comparação de campos é atualizado.

### Tratamento e Limpeza de Dados

Os dados são tratados para:
- Remover duplicatas.
- Padronizar texto para maiúsculas e remover acentos.
- Limpar e padronizar nomes de municípios e países.
- Classificar profissões usando uma tabela de referência.
- Aplicar múltiplas substituições em campos específicos para normalizar categorias e descrições (como faixas etárias e relações de parentesco).

### União e Exportação de Dados

Os dados tratados são combinados com conjuntos de dados de períodos anteriores. Atualmente trabalhamos com dois datasets: um deles contempla o perído de 2014 até os dados mais atuais (usado principalmente para análises temporais) enquanto outro contempla o período de 2020 até os dados mais atuais (usado principalmente para análises de perfis de vítima e suspeito). Isso pois, os dados coletados no momento da denúncia foram alterados diversas vezes ao longo dos anos (ver "Comparação dos campos de cada tabela.xlsx") resultando em diversas incomptibilidades, as quais nos obrigam a trabalhar com esses dois datasets. Portanto, os novos dados inseridos através desse script atualizaram os dois datasets citados.

### Salvamento dos Resultados

Os resultados finais são salvos em arquivos CSV para registros atualizados de todo o período e do período pós-2020. Além disso, a tabela comparativa de nomes de campos é atualizada e exportada em formato Excel.

### Observação

Caso o novo dataset tenha um número de colunas diferentes, ou mesmo as colunas tenham nomes diferentes, a inserção não será realizada! Nesse caso será necessário avaliar manualmente como tratar essa mudança e atualizar o tratamento conforme preciso.


# 1) Imports e funções

In [24]:
import pandas as pd
from unidecode import unidecode

def RemoveAcentosCaixaAlta(celula):
    if isinstance(celula, str):
        return unidecode(celula).upper()
    return celula

def ClassificaProfissao(profissao, arquivoComProfissoes):
    if pd.isna(profissao) or profissao == "":
        return ""
    if profissao == "ESTUDANTE":
        return "ESTUDANTE"
    else:
        for grupo in ['Grupo 1', 'Grupo 2', 'Grupo 3', 'Grupo 4', 'Grupo 5']:
            # Get the radicals of the group, ignoring null values
            radicais = arquivoComProfissoes[grupo].dropna().values
            for radical in radicais:
                if radical in profissao:
                    return grupo.upper()
        return "OUTROS"
    
def PadronizaMunicipio(municipio):
    if isinstance(municipio, str):
        municipioLimpo = municipio.split('|')[-1].strip()
        return municipioLimpo
    return municipio

def PadronizaPais(pais):
    if isinstance(pais, str) and '|' in pais:
        paisLimpo = pais.split('|')[-1].strip()
        return paisLimpo
    return pais

# 2) Reading required files (file names need to be updated as new semesters of data are entered)

In [25]:
# Update these names as necessary
nomeDoNovoArquivo = 'ligue180-primeiro-semestre-2024.csv'
nomeDoArquivoDeTodoPeriodo = '2014 a 2023-2.csv'
nomeDoArquivoDepoisDe2020 = '2020-2 a 2023-2.csv'
nomeDoNovoArquivoDeTodoPeriodo = '2014 a 2024-1.csv'
nomeDoNovoArquivoDepoisDe2020 = '2020-2 a 2024-1.csv'
periodoNovo = '2024-1'

# Try not to change these names
nomeDoArquivoComNomeDasColunas = 'Comparação dos campos de cada tabela.xlsx'
nomeDoArquivoComMunicipios = 'municipios.csv'
nomeDoArquivoComProfissoes = 'profissoes.xlsx'

In [26]:
# Read the new file
novaTabela = pd.read_csv(nomeDoNovoArquivo, sep=";", dtype=str, on_bad_lines='skip')

In [27]:
# Reads the file for the entire period
tabelaDeTodoPeriodo = pd.read_csv(nomeDoArquivoDeTodoPeriodo, sep=",", dtype=str)

In [28]:
# Reads the file with data since 2020
tabelaDepoisDe2020 = pd.read_csv(nomeDoArquivoDepoisDe2020, sep=',', dtype=str)

In [29]:
# Reads the file with the name of the columns to be replaced
tabelaComNomesDasColunas = pd.read_excel(nomeDoArquivoComNomeDasColunas)

In [30]:
# Reads the file with the list of municipalities in Brazil
arquivoComMunicipios = pd.read_csv(nomeDoArquivoComMunicipios, dtype=str, delimiter=';', encoding = "ISO-8859-1")
arquivoComMunicipios['MUNICÍPIO - TOM'] = arquivoComMunicipios['MUNICÍPIO - TOM'].apply(unidecode)

In [31]:
# Read the file with the ENEM profession categories
arquivoComProfissoes = pd.read_excel(nomeDoArquivoComProfissoes, dtype=str)
arquivoComProfissoes = arquivoComProfissoes.map(lambda x: x.upper() if isinstance(x, str) else x)

# 3) Verificação de se a nova tabela manteve o mesmo padrão nos dados fornecidos

In [32]:
padraoAtual = tabelaComNomesDasColunas.iloc[-1, 1:].tolist()
padraoNaNovaTabela = novaTabela.columns.tolist()

if not set(padraoAtual) == set(padraoNaNovaTabela):
    print("A nova tabela mudou o padrão no dados. As seguintes colunas não batem:")

    colunasNaTabelaAtual = set(padraoAtual) - set(padraoNaNovaTabela)
    colunasNaTabelaAntiga = set(padraoNaNovaTabela) - set(padraoAtual)

    if colunasNaTabelaAtual:
        print("Valores na tabela antiga que não estão na nova tabela:", colunasNaTabelaAtual)
    if colunasNaTabelaAntiga:
        print("Valores na nova tabela que não estão na tabela antiga:", colunasNaTabelaAntiga)
else:
    # Update current pattern in field comparison table
    cabecalhoAtual = novaTabela.columns.tolist()
    cabecalhoAtualSeries = pd.Series(cabecalhoAtual)

    primeiraColuna = "Padrão seguido atualmente"
    primeiraColunaSeries = pd.Series([primeiraColuna])

    novoPadraoAtual = pd.concat([primeiraColunaSeries, cabecalhoAtualSeries], ignore_index=True)

    tabelaComNomesDasColunas.iloc[-1] = novoPadraoAtual
        # Insert the column names of the new file into the field comparison table
    parteInicial = tabelaComNomesDasColunas.iloc[:-1]
    ultimaLinha = tabelaComNomesDasColunas.iloc[-1]

    ultimaLinhaInicial = parteInicial.iloc[-1].copy()

    nomePrimeiraColuna = parteInicial.columns[0]
    ultimaLinhaInicial[nomePrimeiraColuna] = periodoNovo

    novaLinhaDF = pd.DataFrame([ultimaLinhaInicial], columns=parteInicial.columns)

    tabelaComNomesDasColunas = pd.concat([parteInicial, novaLinhaDF, ultimaLinha.to_frame().T], ignore_index=True)

    print("A tabela antiga e a tabela nova possuem ainda as mesmas colunas.")
    print(f"Padrão atual:          {set(padraoAtual)}")
    print(f"Padrão na nova tabela: {set(padraoNaNovaTabela)}")

A tabela antiga e a tabela nova possuem ainda as mesmas colunas.
Padrão atual:          {'Frequência', 'Faixa_de_renda_do_suspeito', 'Município_da_vítima', 'Etnia_da_vítima', 'Vínculo_Órgão_PJ_do_suspeito', 'sl_vitima_naturalizado_uf', 'Doença_rara_do_suspeito', 'Canal_de_atendimento', 'Suspeito_Etnia', 'Nacionalidade_da_vítima', 'Grau_de_instrução_do_suspeito', 'Suspeito_nacionalidade', 'Deficiência_do_suspeito', 'Suspeito_preso', 'País_do_suspeito', 'Profissão_do_suspeito', 'Profissão_da_vítima', 'Início_das_violações', 'sl_suspeito_naturalizado_municipio', 'Faixa_etária_da_vítima', 'Orientação_sexual_da_vítima', 'sl_vitima_naturalidade', 'Motivação', 'Religião_da_vítima', 'sl_suspeito_municipio', 'Orientação_sexual_do_suspeito', 'UF', 'sl_quantidade_vitimas', 'Raça_Cor_da_vítima', 'hash', 'Deficiência_relacionada_a_doença_rara', 'sl_vitima_naturalizado_municipio', 'Deficiência_relacionada_a_doença_rara_suspeito', 'suspeito_especificacao_etnia', 'País', 'sl_suspeito_naturalidade', 'G

# 4) Data processing

## 4.1) Update the header of the new table to be able to concatenate with the old data

In [34]:
# Standardizes the name of the columns in the new table, based on the previous tables
novosNomesParaAsColunas = tabelaComNomesDasColunas.iloc[14].tolist()
novosNomesParaAsColunas.pop(0)
novosNomesParaAsColunas = [item for item in novosNomesParaAsColunas if not pd.isna(item)]
novaTabela.columns = novosNomesParaAsColunas

# Remove duplicatas com base na coluna de hash
novaTabela = novaTabela.drop_duplicates(subset='Hash', keep='first')

# Aplica sobre cada célula da tabela a fução que remove acentos e converte para caixa alta
novaTabela = novaTabela.map(RemoveAcentosCaixaAlta)

## 4.2) Filter data only by violence against women (if necessary)

In [11]:
#novaTabela['Grupo vulnerável'].unique()

In [12]:
#novaTabela = novaTabela[novaTabela['Grupo vulnerável'].isin([""])]

## 4.3) Realiza limpezas especificas no novo dataset

In [35]:

novaTabela = novaTabela.astype('string')
for coluna in novaTabela.columns: novaTabela[coluna] = novaTabela[coluna].replace(['NAO SABE INFORMAR', 'N/D', "ATENDIMENTO INTERROMPIDO", "DENUNCIANTE NAO SOUBE INFORMAR", "NAO INFORMADO"], '')
novaTabela = novaTabela.map(lambda x: x.strip() if isinstance(x, str) else x)

################################################################

novaTabela['País'] = novaTabela['País'].apply(PadronizaPais)

novaTabela['UF'] = novaTabela['UF'].str.strip()
novaTabela['UF'] = novaTabela['UF'].mask(~novaTabela['UF'].isin(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]), '')

novaTabela['Município'] = novaTabela['Município'].apply(PadronizaMunicipio)
novaTabela['Município'] = novaTabela['Município'].where(novaTabela['Município'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")

################################################################
novaTabela['Nacionalidade da vítima'] = novaTabela['Nacionalidade da vítima'].apply(PadronizaPais)

novaTabela['País da vítima'] = novaTabela['País da vítima'].apply(PadronizaPais)

novaTabela['UF da vítima'] = novaTabela['UF da vítima'].str.strip()
novaTabela['UF da vítima'] = novaTabela['UF da vítima'].mask(~novaTabela['UF da vítima'].isin(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]), '')

novaTabela['Município da vítima'] = novaTabela['Município da vítima'].apply(PadronizaMunicipio)
novaTabela['Município da vítima'] = novaTabela['Município da vítima'].where(novaTabela['Município da vítima'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")

novaTabela['sl vitima naturalizado uf'] = novaTabela['sl vitima naturalizado uf'].str.strip()
novaTabela['sl vitima naturalizado uf'] = novaTabela['sl vitima naturalizado uf'].mask(~novaTabela['sl vitima naturalizado uf'].isin(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]), '')

novaTabela['sl vitima naturalizado municipio'] = novaTabela['sl vitima naturalizado municipio'].apply(PadronizaMunicipio)
novaTabela['sl vitima naturalizado municipio'] = novaTabela['sl vitima naturalizado municipio'].where(novaTabela['sl vitima naturalizado municipio'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")

################################################################

novaTabela['Suspeito nacionalidade'] = novaTabela['Nacionalidade da vítima'].apply(PadronizaPais)

novaTabela['País do suspeito'] = novaTabela['País do suspeito'].apply(PadronizaPais)

novaTabela['UF do suspeito'] = novaTabela['UF do suspeito'].str.strip()
novaTabela['UF do suspeito'] = novaTabela['UF do suspeito'].mask(~novaTabela['UF do suspeito'].isin(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]), '')

novaTabela['Município do suspeito'] = novaTabela['Município do suspeito'].apply(PadronizaMunicipio)
novaTabela['Município do suspeito'] = novaTabela['Município do suspeito'].where(novaTabela['Município do suspeito'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")

novaTabela['sl suspeito naturalizado uf'] = novaTabela['sl suspeito naturalizado uf'].str.strip()
novaTabela['sl suspeito naturalizado uf'] = novaTabela['sl suspeito naturalizado uf'].mask(~novaTabela['sl suspeito naturalizado uf'].isin(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]), '')

novaTabela['sl suspeito naturalizado municipio'] = novaTabela['sl suspeito naturalizado municipio'].apply(PadronizaMunicipio)
novaTabela['sl suspeito naturalizado municipio'] = novaTabela['sl suspeito naturalizado municipio'].where(novaTabela['sl suspeito naturalizado municipio'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")

################################################################

novaTabela['Sexo da vítima'] = novaTabela['Sexo da vítima'].mask(~novaTabela['Sexo da vítima'].isin(['FEMININO', 'MASCULINO']), '')
novaTabela['Sexo do suspeito'] = novaTabela['Sexo do suspeito'].mask(~novaTabela['Sexo do suspeito'].isin(['FEMININO', 'MASCULINO']), '')

novaTabela['Faixa etária da vítima'] = novaTabela['Faixa etária da vítima'].replace(['10 ANOS', '11 ANOS'], '10 A 11 ANOS')
novaTabela['Faixa etária da vítima'] = novaTabela['Faixa etária da vítima'].replace(['12 ANOS', '13 ANOS', '14 ANOS'], '12 A 14 ANOS')
novaTabela['Faixa etária do suspeito'] = novaTabela['Faixa etária do suspeito'].replace(['15 ANOS', '16 ANOS', '17 ANOS'], '15 A 17 ANOS')
novaTabela['Faixa etária do suspeito'] = novaTabela['Faixa etária do suspeito'].replace(['12 ANOS', '13 ANOS', '14 ANOS'], '12 A 14 ANOS')

novaTabela['Deficiência da vítima'] = novaTabela['Deficiência da vítima'].replace(['NAO'], 'NAO TEM DEFICIENCIA')
novaTabela['Deficiência do suspeito'] = novaTabela['Deficiência do suspeito'].replace(['NAO'], 'NAO TEM DEFICIENCIA')

novaTabela['Doença rara da vítima'] = novaTabela['Doença rara do suspeito'].replace(['NAO'], 'NAO TEM DOENCA RARA')
novaTabela['Doença rara do suspeito'] = novaTabela['Doença rara do suspeito'].replace(['NAO'], 'NAO TEM DOENCA RARA')

novaTabela['sl vitima cadastro'] = novaTabela['sl vitima cadastro'].replace(['COMUNIDADE', 'FAMILIA'], 'COMUNIDADE/FAMILIA')
novaTabela['Natureza Jurídica do Suspeito'] = novaTabela['Natureza Jurídica do Suspeito'].replace(['COMUNIDADE', 'FAMILIA'], 'COMUNIDADE/FAMILIA')

novaTabela['Orientação sexual da vítima'] = novaTabela['Orientação sexual da vítima'].replace(['HOMOSSEXUAL.LESBICA', 'HOMOSSEXUAL.GAY'], 'HOMOSSEXUAL')
novaTabela['Orientação sexual do suspeito'] = novaTabela['Orientação sexual do suspeito'].replace(['HOMOSSEXUAL.LESBICA', 'HOMOSSEXUAL.GAY'], 'HOMOSSEXUAL')

novaTabela['Grau de instrução da vítima'] = novaTabela['Grau de instrução da vítima'].replace(['ANALFABETO/SEM INSTRUCAO'], 'ANALFABETO')
novaTabela['Grau de instrução do suspeito'] = novaTabela['Grau de instrução do suspeito'].replace(['ANALFABETO/SEM INSTRUCAO'], 'ANALFABETO')

novaTabela['Religião da vítima'] = novaTabela['Religião da vítima'].replace(['ANALFABETO/SEM INSTRUCAO'], 'ANALFABETO')
novaTabela['Religião do suspeito'] = novaTabela['Religião do suspeito'].replace(['ANALFABETO/SEM INSTRUCAO'], 'ANALFABETO')

novaTabela['Etnia da vítima'] = novaTabela['Etnia da vítima'].replace(['PRETO', 'PARDO', 'PARDA'], '')
novaTabela['Etnia da vítima'] = novaTabela['Etnia da vítima'].replace(['COMUNIDADES QUILOMBOLAS'], 'QUILOMBOLAS')
novaTabela['Etnia da vítima'] = novaTabela['Etnia da vítima'].replace(['POVOS INDIGENAS'], 'INDIGENA')
novaTabela['Etnia da vítima'] = novaTabela['Etnia da vítima'].replace(['DEMAIS COMUNIDADES TRADICIONAIS'], 'POVOS E COMUNIDADES TRADICIONAIS')
novaTabela['Etnia da vítima'] = novaTabela['Etnia da vítima'].replace(['POVOS CIGANOS'], 'CIGANA')
novaTabela['Etnia do Suspeito'] = novaTabela['Etnia do Suspeito'].replace(['PRETO', 'PARDO', 'PARDA'], '')
novaTabela['Etnia do Suspeito'] = novaTabela['Etnia do Suspeito'].replace(['COMUNIDADES QUILOMBOLAS'], 'QUILOMBOLAS')
novaTabela['Etnia do Suspeito'] = novaTabela['Etnia do Suspeito'].replace(['POVOS INDIGENAS'], 'INDIGENA')
novaTabela['Etnia do Suspeito'] = novaTabela['Etnia do Suspeito'].replace(['DEMAIS COMUNIDADES TRADICIONAIS'], 'POVOS E COMUNIDADES TRADICIONAIS')
novaTabela['Etnia do Suspeito'] = novaTabela['Etnia do Suspeito'].replace(['POVOS CIGANOS'], 'CIGANO')

novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['ALUNO (A)'], 'ALUNO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['AMIGO (A)'], 'AMIGO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['AVO(O)'], 'AVO')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['COMPANHEIRO (A)'], 'COMPANHEIRO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['CUIDADOR (A)'], 'CUIDADOR(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['CUNHADO (A)'], 'CUNHADO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['EMPREGADO(HIERARQUICAMENTE INFERIOR)'], 'EMPREGADO (HIERARQUICAMENTE INFERIOR)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['EMPREGADOR/ PATRAO(HIERARQUICAMENTE SUPERIOR)', 'EMPREGADOR/PATRAO(HIERARQUICAMENTE SUPERIOR)'], 'EMPREGADOR/PATRAO (HIERARQUICAMENTE SUPERIOR)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['ESPOSA', 'MARIDO'], 'ESPOSO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['EX-COMPANHEIRO (A)'], 'EX-COMPANHEIRO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['EX-ESPOSA', 'EX-MARIDO'], 'EX-ESPOSO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['EX-NAMORADO (A)'], 'EX-NAMORADO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['FILHO (A)'], 'FILHO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['IRMAO (A)'], 'IRMAO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['LIDER RELIGIOSO'], 'LIDER RELIGIOSO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['MORA NA MESMA RESIDENCIA MAS NAO E FAMILIAR', 'MOROU NA MESMA RESIDENCIA MAS NAO E FAMILIAR'], 'MORA/MOROU NA MESMA RESIDENCIA MAS NAO E FAMILIAR')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['FAMILIARES', 'OUTROS FAMILIARES', 'OUTROS PROFISSIONAIS DA EDUCACAO'], 'OUTROS')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['PRESTADOR DE SERVICO'], 'PRESTADOR(A) DE SERVICO')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['TIO (A)'], 'TIO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['TREINADOR/TECNICO'], 'TREINADOR(A)/TECNICO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['TRISAVO(O)'], 'TRISAVO')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['VIZINHO (A)'], 'VIZINHO(A)')

novaTabela['Violações'] = novaTabela['Violações'].str.replace(';', '', regex=False)

substituicoes = {
    'NAMORADO(A)': 'PARCEIRO(A) ATUAL',
    'COMPANHEIRO (A)': 'PARCEIRO(A) ATUAL',
    'COMPANHEIRO(A)': 'PARCEIRO(A) ATUAL',
    'ESPOSA': 'PARCEIRO(A) ATUAL',
    'ESPOSA(O)': 'PARCEIRO(A) ATUAL',
    'MARIDO': 'PARCEIRO(A) ATUAL',
    'EX-COMPANHEIRO (A)': 'EX-PARCEIRO(A)',
    'EX-ESPOSA': 'EX-PARCEIRO(A)',
    'EX-MARIDO': 'EX-PARCEIRO(A)',
    'EX-NAMORADO (A)': 'EX-PARCEIRO(A)',
    'EX-COMPANHEIRO(A)': 'EX-PARCEIRO(A)',
    'EX-NAMORADO(A)': 'EX-PARCEIRO(A)',
    'EX-ESPOSA(O)': 'EX-PARCEIRO(A)',
    'ALUNO (A)': 'ALUNO(A)',
    'AMIGO (A)': 'AMIGO(A)',
    'CUIDADOR (A)': 'CUIDADOR(A)',
    'CUNHADO (A)': 'CUNHADO(A)',
    'EMPREGADO(HIERARQUICAMENTE INFERIOR)': 'EMPREGADO (HIERARQUICAMENTE INFERIOR)',
    'EMPREGADOR/ PATRAO(HIERARQUICAMENTE SUPERIOR)': 'EMPREGADOR/PATRAO (HIERARQUICAMENTE SUPERIOR)',
    'EMPREGADOR/PATRAO(HIERARQUICAMENTE SUPERIOR)': 'EMPREGADOR/PATRAO (HIERARQUICAMENTE SUPERIOR)',
    'FILHO (A)': 'FILHO(A)',
    'IRMAO (A)': 'IRMAO(A)',
    'LIDER RELIGIOSO': 'LIDER RELIGIOSO(A)',
    'MORA NA MESMA RESIDENCIA MAS NAO E FAMILIAR': 'MORA/MOROU NA MESMA RESIDENCIA MAS NAO E FAMILIAR',
    'MOROU NA MESMA RESIDENCIA MAS NAO E FAMILIAR': 'MORA/MOROU NA MESMA RESIDENCIA MAS NAO E FAMILIAR',
    'FAMILIARES': 'OUTROS',
    'OUTROS FAMILIARES': 'OUTROS',
    'OUTROS PROFISSIONAIS DA EDUCACAO': 'OUTROS',
    'PRESTADOR DE SERVICO': 'PRESTADOR(A) DE SERVICO',
    'TIO (A)': 'TIO(A)',
    'TREINADOR/TECNICO': 'TREINADOR(A)/TECNICO(A)',
    'TRISAVO(O)': 'TRISAVO(O)',
    'VIZINHO (A)': 'VIZINHO(A)'
}
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(substituicoes)

substituicoes = {
    '1 A 3 SM': '1 A 3 SM',
    '3 A 5 SM': '3 A 5 SM',
    '5 A 15 SM': '5 A 15 SM',
    '6 A 15 SM': '5 A 15 SM',
    '8 A 15 SM': '5 A 15 SM',
    'ACIMA DE 15 SM': 'ACIMA DE 15 SM',
    'ATE 1 SM': 'ATE 1 SM',
    'ATE 1/2 SALARIO MINIMO': 'ATE 1/2 SM',
    'MAIS DE 1 A 2 SALARIOS MINIMOS': '1 A 3 SM',
    'MAIS DE 1/2 A 1 SALARIO MINIMO': 'ATE 1 SM',
    'MAIS DE 10 A 20 SALARIOS MINIMOS': '10 A 20 SM',
    'MAIS DE 2 A 5 SALARIOS MINIMOS': '3 A 5 SM',
    'MAIS DE 20 SALARIOS MINIMOS': 'MAIS DE 20 SM',
    'MAIS DE 5 A 10 SALARIOS MINIMOS': '5 A 15 SM',
    'SEM RENDIMENTO': 'SEM RENDIMENTO'
}

novaTabela['Faixa de renda da vítima'] = novaTabela['Faixa de renda da vítima'].replace(substituicoes)

novaTabela = novaTabela.drop('Hash', axis=1)
novaTabela = novaTabela.drop('Grupo vulnerável', axis=1)

# Standardize according to ENEM data dictionary
novaTabela['Profissão da vítima'] = novaTabela['Profissão da vítima'].apply(lambda profissao: ClassificaProfissao(profissao, arquivoComProfissoes))
novaTabela['Profissão do suspeito'] = novaTabela['Profissão do suspeito'].apply(lambda profissao: ClassificaProfissao(profissao, arquivoComProfissoes))

In [ ]:
def agrupar_violacoes(novaTabela):
        # Defining the categories of violence and their associated words
    violencias = {
        "violencia_fisica": [
            "LESAO CORPORAL", "LESAO CORPORAL LEVE", "LESAO CORPORAL GRAVE", "LESAO CORPORAL GRAVISSIMA",
            "AGRESSAO OU VIAS DE FATO", "MAUS TRATOS", "TORTURA FISICA", "FEMINICIDIO", "VIOLENCIA OBSTETRICA",
            "EXPOSICAO DE RISCO A SAUDE", "HOMICIDIO", "VIOLENCIA POLICIAL", "DIREITOS SOCIAIS",
            "AGRESSAO OU VIAS DE FATO, INTEGRIDADE>FISICA>EXPOSICAO DE RISCO A SAUDE",
            "AGRESSAO OU VIAS DE FATO, INTEGRIDADE>FISICA>LESAO CORPORAL", "QUEIMADURA", "FRATURA", "CONTUSAO",
            "ESFAQUEAMENTO", "TIRO"
        ],
        "violencia_psicologica": [
            "AMEACA", "CARCERE PRIVADO", "VIOLENCIA_PSICOLOGICA", "TORTURA PSIQUICA", "LIBERDADE DE IR VIR PERMANECER",
            "AUTONOMIA DE VONTADE", "INSUBSISTENCIA AFETIVA", "ASSEDIO MORAL", "SEQUESTRO", "BULLYING", "ALIENACAO PARENTAL",
            "PERSEGUICAO", "VIOLENCIA VERBAL", "INTIMIDACAO", "ISOLAMENTO SOCIAL"
        ],
        "violencia_sexual": [
            "ESTUPRO", "ASSSEDIO SEXUAL (TRABALHO)", "VIOLENCIA SEXUAL", "ASSEDIO SEXUAL", "ABUSO SEXUAL PSIQUICO",
            "ABUSO SEXUAL FISICO", "EXPLORACAO SEXUAL", "ABUSO / IMPORTUNACAO SEXUAL FISICA",
            "ABUSO / IMPORTUNACAO SEXUAL PSIQUICA", "ESTUPRO DE VULNERAVEL", "ESTUPRO VIRTUAL",
            "VIOLENCIA SEXUAL MEDIANTE FRAUDE", "ATENTADO AO PUDOR", "EXPOSICAO INDEVIDA", "VOYEURISMO", "ABUSO ONLINE",
            "ABUSO INFANTIL", "TRAFICO DE PESSOAS PARA FINS SEXUAIS"
        ],
        "violencia_patrimonial": [
            "VIOLENCIA PATRIMONIAL", "INSUBSISTENCIA MATERIAL", "PATRIMONIAL>INDIVIDUAL", "PATRIMONIAL>COLETIVO",
            "PATRIMONIAL>CULTURAL", "FALTA DE ACESSIBILIDADE", "FURTO", "ROUBO", "DESTRUICAO DE PROPRIEDADE",
            "COBRANCA ABUSIVA", "INTERFERENCIA ECONOMICA"
        ],
        "violencia_moral": [
            "INJURIA", "DIFAMACAO", "CALUNIA", "CONSTRANGIMENTO", "DISCRIMINACAO", "EXPOSICAO", "HUMILHACAO",
            "DESCRIMINACAO", "FOFOCA", "ATAQUE A REPUTACAO", "LINGUAGEM PEJORATIVA"
        ],
        "outro": [
            "OUTROS", "OUTRO", "DISCRIMINACAO", "ABANDONO", "NEGLIGENCIA", "VIOLACAO DE PRIVACIDADE",
            "VIOLACAO DE DIREITOS HUMANOS"
        ]
    }

    def determinar_categorias(violacao):
        categorias = []
        if isinstance(violacao, str):
            for categoria, palavras in violencias.items():
                if any(palavra in violacao for palavra in palavras):
                    categorias.append(categoria)
        return categorias if categorias else ["outro"]

    novaTabela['categorias'] = novaTabela['Violações'].apply(determinar_categorias)

    contagem_categorias = {categoria: novaTabela['categorias'].apply(lambda x: categoria in x).sum() for categoria in violencias.keys()}
    contagem_df = pd.DataFrame.from_dict(contagem_categorias, orient='index', columns=['count'])
    contagem_df = contagem_df.sort_values(by='count', ascending=False)
    
    return contagem_df

In [ ]:
novaTabela = agrupar_violacoes(novaTabela)

# 5) Merges the new dataset with the data from 2020 onwards and exports the file

In [36]:
# Inserted the new data into the table with data after 2020
novaTabelaDepoisDe2020 = pd.concat([novaTabela, tabelaDepoisDe2020], ignore_index=True)

In [37]:
# Save the unified table
novaTabelaDepoisDe2020.to_csv(nomeDoNovoArquivoDepoisDe2020, index=False)

# 6) Merges the new dataset with the data from 2014 to date, processes the merged data and exports the file

In [38]:
# Insert the new data into the table with data from the whole period
novaTabelaDeTodoPeriodo = pd.concat([novaTabela, tabelaDeTodoPeriodo], ignore_index=True)

In [39]:
substituicoes = {
    'TELEFONICO': 'ATENDIMENTO TELEFÔNICO',
    'CENTRAL DE ATENDIMENTO': 'ATENDIMENTO TELEFÔNICO',
    'DENUNCIA - ATENDIMENTO TELEFONICO': 'ATENDIMENTO TELEFÔNICO',
    'ATENDIMENTO TELEFONICO': 'ATENDIMENTO TELEFÔNICO',
    'CENTRAL DE ATENDIMENTO SEPPIR': 'ATENDIMENTO TELEFÔNICO',
    
    'WHATSAPP': 'APLICATIVOS/ONLINE',
    'TELEGRAM': 'APLICATIVOS/ONLINE',
    'APLICATIVO': 'APLICATIVOS/ONLINE',
    'APLICATIVO/APP': 'APLICATIVOS/ONLINE',
    'MOBILE DH': 'APLICATIVOS/ONLINE',
    'PROTEJA BRASIL - APLICATIVO': 'APLICATIVOS/ONLINE',
    'MOBILE SABE': 'APLICATIVOS/ONLINE',
    'MOBILE GDF': 'APLICATIVOS/ONLINE',
    
    'E-MAIL': 'E-MAIL',
    'E-MAIL: LIGUE180@MDH.GOV.BR': 'E-MAIL',
    'DENUNCIA - ATENDIMENTO E-MAIL': 'E-MAIL',
    
    'WEBCHAT': 'WEB/ONLINE CHAT',
    'PORTAL': 'WEB/ONLINE CHAT',
    'E-OUV': 'WEB/ONLINE CHAT',
    'DENUNCIA WEB': 'WEB/ONLINE CHAT',
    
    'PRESENCIAL': 'CORRESPONDÊNCIA/PRESENCIAL',
    'CARTA': 'CORRESPONDÊNCIA/PRESENCIAL',
    'OFICIO': 'CORRESPONDÊNCIA/PRESENCIAL',
    'DENUNCIA - ATENDIMENTO PRESENCIAL': 'CORRESPONDÊNCIA/PRESENCIAL',
    'DENUNCIA - ATENDIMENTO OFICIO': 'CORRESPONDÊNCIA/PRESENCIAL',
    'DENUNCIA - ATENDIMENTO CARTA': 'CORRESPONDÊNCIA/PRESENCIAL',
    
    'VIDEOCHAMADA': 'VÍDEO/OUTROS',
    'PROCESSO SEI': 'VÍDEO/OUTROS',
    'AUTUACAO DE OFICIO': 'VÍDEO/OUTROS',
}
novaTabelaDeTodoPeriodo['Canal de atendimento'] = novaTabelaDeTodoPeriodo['Canal de atendimento'].replace(substituicoes)

substituicoes = {
    'A PROPRIA VITIMA': 'VITIMA',
    'COMPANHEIRO': 'PARCEIRO/COMPANHEIRO',
    'COMPANHEIRA (LESBICA)': 'PARCEIRO/COMPANHEIRO',
    'EX COMPANHEIRO': 'EX-PARCEIRO/COMPANHEIRO',
    'EX COMPANHEIRA (LESBICA)': 'EX-PARCEIRO/COMPANHEIRO',
    'CONJUGE': 'PARCEIRO/COMPANHEIRO',
    'NAMORADO': 'PARCEIRO/COMPANHEIRO',
    'NAMORADA (LESBICA)': 'PARCEIRO/COMPANHEIRO',
    'EX NAMORADO': 'EX-PARCEIRO/COMPANHEIRO',
    'EX NAMORADA (LESBICA)': 'EX-PARCEIRO/COMPANHEIRO',
    'ANONIMO': 'DESCONHECIDO',
    
    'OUTRO(A)': 'OUTROS',
    'PROPRIETARIO(A) DO IMOVEL EM QUE A VITIMA RESIDE': 'OUTROS',
    'TERCEIRO': 'OUTROS',
    'O PROPRIO AGRESSOR': 'OUTROS',
    'PESSOA COM DEFICIENCIA MENTAL': 'OUTROS',
    'PESSOA EM SOFRIMENTO PSIQUICO': 'OUTROS',
}
novaTabelaDeTodoPeriodo['Denunciante'] = novaTabelaDeTodoPeriodo['Denunciante'].replace(substituicoes)

substituicoes = {
    '1 A 3 SALARIOS MINIMOS': '1 A 3 SM',
    '1 A 3 SM': '1 A 3 SM',
    '3 A 5 SALARIOS MINIMOS': '3 A 5 SM',
    '3 A 5 SM': '3 A 5 SM',
    '5 A 15 SALARIOS MINIMOS': '5 A 15 SM',
    '5 A 15 SM': '5 A 15 SM',
    'ACIMA DE  15 SALARIOS MINIMOS': 'ACIMA DE 15 SM',
    'ACIMA DE 15 SM': 'ACIMA DE 15 SM',
    'ATE 1 SALARIO MINIMO': 'ATE 1 SM',
    'ATE 1 SM': 'ATE 1 SM',
    'ATE 1/2 SM': 'ATE 1/2 SM',
    '10 A 20 SM': '10 A 20 SM',
    'MAIS DE 20 SM': 'MAIS DE 20 SM',
    'SEM RENDIMENTO': 'SEM RENDIMENTO'
}
novaTabelaDeTodoPeriodo['Faixa de renda da vítima'] = novaTabelaDeTodoPeriodo['Faixa de renda da vítima'].replace(substituicoes)
novaTabelaDeTodoPeriodo['Faixa de renda do suspeito'] = novaTabelaDeTodoPeriodo['Faixa de renda do suspeito'].replace(substituicoes)

substituicoes = {
    '0 A 3 ANOS': '0 A 3 ANOS',
    '01 ANO': '0 A 3 ANOS',
    '02 ANOS': '0 A 3 ANOS',
    '03 ANOS': '0 A 3 ANOS',
    'MENOS DE 01 ANO': '0 A 3 ANOS',
    
    '04 ANOS': '4 A 7 ANOS',
    '05 ANOS': '4 A 7 ANOS',
    '06 ANOS': '4 A 7 ANOS',
    '07 ANOS': '4 A 7 ANOS',
    '4 A 7 ANOS': '4 A 7 ANOS',
    
    '08 ANOS': '8 A 11 ANOS',
    '10 A 11 ANOS': '8 A 11 ANOS',
    '8 A 11 ANOS': '8 A 11 ANOS',
    'ATE 9 ANOS': '8 A 11 ANOS',
    
    '12 A 14 ANOS': '12 A 14 ANOS',
    'ENTRE 10 E 14 ANOS': '12 A 14 ANOS',
    
    '15 ANOS': '15 A 17 ANOS',
    '16 ANOS': '15 A 17 ANOS',
    '17 ANOS': '15 A 17 ANOS',
    '15 A 17 ANOS': '15 A 17 ANOS',
    
    '18 A 19 ANOS': '18 A 24 ANOS',
    '18 A 24 ANOS': '18 A 24 ANOS',
    '20 A 24 ANOS': '18 A 24 ANOS',
    'ENTRE 15 E 19 ANOS': '18 A 24 ANOS',
    'ENTRE 20 E 24 ANOS': '18 A 24 ANOS',
    
    '25 A 29 ANOS': '25 A 29 ANOS',
    '25 A 30 ANOS': '25 A 29 ANOS',
    'ENTRE 25 E 29 ANOS': '25 A 29 ANOS',
    
    '30 A 34 ANOS': '30 A 34 ANOS',
    '31 A 35 ANOS': '30 A 34 ANOS',
    'ENTRE 30 E 34 ANOS': '30 A 34 ANOS',
    
    '35 A 39 ANOS': '35 A 39 ANOS',
    '36 A 40 ANOS': '35 A 39 ANOS',
    'ENTRE 35 E 39 ANOS': '35 A 39 ANOS',
    
    '40 A 44 ANOS': '40 A 44 ANOS',
    '41 A 45 ANOS': '40 A 44 ANOS',
    'ENTRE 40 E 44 ANOS': '40 A 44 ANOS',
    
    '45 A 49 ANOS': '45 A 49 ANOS',
    '46 A 50 ANOS': '45 A 49 ANOS',
    'ENTRE 45 E 49 ANOS': '45 A 49 ANOS',
    
    '50 A 54 ANOS': '50 A 54 ANOS',
    '51 A 55 ANOS': '50 A 54 ANOS',
    'ENTRE 50 E 54 ANOS': '50 A 54 ANOS',
    
    '55 A 59 ANOS': '55 A 59 ANOS',
    '56 A 60 ANOS': '55 A 59 ANOS',
    'ENTRE 55 E 59 ANOS': '55 A 59 ANOS',
    
    '60 A 64 ANOS': '60 A 64 ANOS',
    '61 A 65 ANOS': '60 A 64 ANOS',
    'ENTRE 60 E 64 ANOS': '60 A 64 ANOS',
    
    '65 A 69 ANOS': '65 A 69 ANOS',
    '66 A 70 ANOS': '65 A 69 ANOS',
    'ENTRE 65 E 69 ANOS': '65 A 69 ANOS',
    
    '70 A 74 ANOS': '70 A 74 ANOS',
    '71 A 75 ANOS': '70 A 74 ANOS',
    'MAIOR DE 70 ANOS': '70 A 74 ANOS',
    
    '75 A 79 ANOS': '75 A 79 ANOS',
    '76 A 80 ANOS': '75 A 79 ANOS',
    
    '80 A 84 ANOS': '80 A 84 ANOS',
    '81 A 85 ANOS': '80 A 84 ANOS',
    
    '85 A 89 ANOS': '85 A 89 ANOS',
    '85 A 90 ANOS': '85 A 89 ANOS',
    
    '91 ANOS OU MAIS': '90 ANOS OU MAIS',
    
    'CRIANCA/ADOLESCENTE IDADE NAO INFORMADA': 'CRIANCA/ADOLESCENTE IDADE NAO INFORMADA',
    'IDOSO IDADE NAO INFORMADA': 'IDOSO IDADE NAO INFORMADA',
    'NASCITURO': 'NASCITURO',
}
novaTabelaDeTodoPeriodo['Faixa etária da vítima'] = novaTabelaDeTodoPeriodo['Faixa etária da vítima'].replace(substituicoes)

substituicoes = {
    '0 A 3 ANOS': '0 A 3 ANOS',
    '00 A 1 ANO': '0 A 3 ANOS',
    'RECEM-NASCIDO': '0 A 3 ANOS',
    
    '02 A 04 ANOS': '4 A 7 ANOS',
    '4 A 7 ANOS': '4 A 7 ANOS',
    
    '05 A 09 ANOS': '5 A 9 ANOS',
    'ATE 9 ANOS': '5 A 9 ANOS',
    
    '8 A 11 ANOS': '8 A 11 ANOS',
    '10 A 11 ANOS': '8 A 11 ANOS',
    
    '12 A 14 ANOS': '12 A 14 ANOS',
    'ENTRE 10 E 14 ANOS': '12 A 14 ANOS',
    
    '15 A 17 ANOS': '15 A 17 ANOS',
    'ENTRE 15 E 19 ANOS': '15 A 17 ANOS',
    
    '18 A 19 ANOS': '18 A 24 ANOS',
    '18 A 24 ANOS': '18 A 24 ANOS',
    '20 A 24 ANOS': '18 A 24 ANOS',
    'ENTRE 20 E 24 ANOS': '18 A 24 ANOS',
    
    '25 A 29 ANOS': '25 A 29 ANOS',
    '25 A 30 ANOS': '25 A 29 ANOS',
    'ENTRE 25 E 29 ANOS': '25 A 29 ANOS',
    
    '30 A 34 ANOS': '30 A 34 ANOS',
    '31 A 35 ANOS': '30 A 34 ANOS',
    'ENTRE 30 E 34 ANOS': '30 A 34 ANOS',
    
    '35 A 39 ANOS': '35 A 39 ANOS',
    '36 A 40 ANOS': '35 A 39 ANOS',
    'ENTRE 35 E 39 ANOS': '35 A 39 ANOS',
    
    '40 A 44 ANOS': '40 A 44 ANOS',
    '41 A 45 ANOS': '40 A 44 ANOS',
    'ENTRE 40 E 44 ANOS': '40 A 44 ANOS',
    
    '45 A 49 ANOS': '45 A 49 ANOS',
    '46 A 50 ANOS': '45 A 49 ANOS',
    'ENTRE 45 E 49 ANOS': '45 A 49 ANOS',
    
    '50 A 54 ANOS': '50 A 54 ANOS',
    '51 A 55 ANOS': '50 A 54 ANOS',
    'ENTRE 50 E 54 ANOS': '50 A 54 ANOS',
    
    '55 A 59 ANOS': '55 A 59 ANOS',
    '56 A 60 ANOS': '55 A 59 ANOS',
    'ENTRE 55 E 59 ANOS': '55 A 59 ANOS',
    
    '60 A 64 ANOS': '60 A 64 ANOS',
    '61 A 65 ANOS': '60 A 64 ANOS',
    'ENTRE 60 E 64 ANOS': '60 A 64 ANOS',
    
    '65 A 69 ANOS': '65 A 69 ANOS',
    '66 A 70 ANOS': '65 A 69 ANOS',
    'ENTRE 65 E 69 ANOS': '65 A 69 ANOS',
    
    '70 A 74 ANOS': '70 A 74 ANOS',
    '71 A 75 ANOS': '70 A 74 ANOS',
    'MAIOR DE 70 ANOS': '70 A 74 ANOS',
    
    '75 A 79 ANOS': '75 A 79 ANOS',
    '76 A 80 ANOS': '75 A 79 ANOS',
    
    '80 A 84 ANOS': '80 A 84 ANOS',
    '80 ANOS OU MAIS': '80 A 84 ANOS',
    '81 A 85 ANOS': '80 A 84 ANOS',
    
    '85 A 89 ANOS': '85 A 89 ANOS',
    '85 A 90 ANOS': '85 A 89 ANOS',
    
    '90+': '90 ANOS OU MAIS',
    '91 ANOS OU MAIS': '90 ANOS OU MAIS',
    
    'CRIANCA/ADOLESCENTE IDADE NAO INFORMADA': 'CRIANCA/ADOLESCENTE IDADE NAO INFORMADA',
    'IDOSO IDADE NAO INFORMADA': 'IDOSO IDADE NAO INFORMADA',
    'NASCITURO': 'NASCITURO'
}
novaTabelaDeTodoPeriodo['Faixa etária do suspeito'] = novaTabelaDeTodoPeriodo['Faixa etária do suspeito'].replace(substituicoes)

substituicoes = {
    'DIARIAMENTE': 'DIARIAMENTE',
    'OCORRE TODOS OS DIAS': 'DIARIAMENTE',
    'TODA MANHA': 'DIARIAMENTE',
    'TODA TARDE': 'DIARIAMENTE',
    
    'SEMANALMENTE': 'SEMANALMENTE',
    'OCORRE ALGUMAS VEZES NA SEMANA': 'SEMANALMENTE',
    
    'QUINZENALMENTE': 'QUINZENALMENTE',
    
    'MENSALMENTE': 'MENSALMENTE',
    'OCORRE ALGUMAS VEZES NO DECORRER DO MES': 'MENSALMENTE',
    
    'OCASIONALMENTE': 'ANUALMENTE/OCASIONALMENTE',
    'OCORRE ALGUMAS VEZES NO DECORRER DO ANO': 'ANUALMENTE/OCASIONALMENTE',
    
    'OCORREU UMA VEZ': 'UNICA OCORRENCIA',
    'UNICA OCORRENCIA': 'UNICA OCORRENCIA',
    'UNICA VEZ': 'UNICA OCORRENCIA',
    
    'OUTRO': 'OUTRO',
}
novaTabelaDeTodoPeriodo['Frequência'] = novaTabelaDeTodoPeriodo['Frequência'].replace(substituicoes)

substituicoes = {
    'ANALFABETA(O)': 'ANALFABETO',
    'ANALFABETO': 'ANALFABETO',
    'ANALFABETO(A)': 'ANALFABETO',
    
    'ENSINO FUNDAMENTAL': 'ENSINO FUNDAMENTAL',
    'ENSINO FUNDAMENTAL COMPLETO': 'ENSINO FUNDAMENTAL',
    'ENSINO FUNDAMENTAL INCOMPLETO': 'ENSINO FUNDAMENTAL',
    
    'ENSINO MEDIO': 'ENSINO MEDIO',
    'ENSINO MEDIO COMPLETO': 'ENSINO MEDIO',
    'ENSINO MEDIO INCOMPLETO': 'ENSINO MEDIO',
    
    'ENSINO SUPERIOR': 'ENSINO SUPERIOR',
    'ENSINO SUPERIOR COMPLETO': 'ENSINO SUPERIOR',
    'ENSINO SUPERIOR INCOMPLETO': 'ENSINO SUPERIOR',
    'SUPERIOR COMPLETO': 'ENSINO SUPERIOR',
    'SUPERIOR INCOMPLETO': 'ENSINO SUPERIOR',
    
    'POS GRADUACAO': 'POS-GRADUACAO',
    'POS-DOUTORADO': 'POS-GRADUACAO',
    'POS-GRADUACAO': 'POS-GRADUACAO',
    'ESPECIALIZACAO': 'POS-GRADUACAO',
    'MESTRADO': 'POS-GRADUACAO',
    'DOUTORADO': 'POS-GRADUACAO',
    
    'NIVEL TECNICO': 'NIVEL TECNICO',
    
    'CRIANCA/ADOLESCENTE QUE NAO FREQUENTA INSTITUICAO DE ENSINO': 'CRIANCA/ADOLESCENTE QUE NAO FREQUENTA INSTITUICAO DE ENSINO'
}
novaTabelaDeTodoPeriodo['Grau de instrução da vítima'] = novaTabelaDeTodoPeriodo['Grau de instrução da vítima'].replace(substituicoes)
novaTabelaDeTodoPeriodo['Grau de instrução do suspeito'] = novaTabelaDeTodoPeriodo['Grau de instrução do suspeito'].replace(substituicoes)

substituicoes = {
    'VIA PUBLICA': 'VIA PUBLICA',

    'CASA DE FAMILIARES': 'CASA DE AMIGOS/FAMILIARES',
    'CASA DE FAMILIAR': 'CASA DE AMIGOS/FAMILIARES',
    'CASA DE AMIGOS OU FAMILIARES': 'CASA DE AMIGOS/FAMILIARES',
    'CASA DE TERCEIRO': 'CASA DE AMIGOS/FAMILIARES',

    'CASA DA VITIMA': 'CASA DA VITIMA/SUSPEITO',
    'CASA DA VITIMA/AGRESSOR': 'CASA DA VITIMA/SUSPEITO',
    'CASA DO SUSPEITO': 'CASA DA VITIMA/SUSPEITO',
    'CASA VITIMA': 'CASA DA VITIMA/SUSPEITO',
    'CASA ONDE RESIDE A VITIMA E O SUSPEITO': 'CASA DA VITIMA/SUSPEITO',

    'AMBIENTE VIRTUAL': 'AMBIENTE VIRTUAL',
    'AMBIENTE VIRTUAL (NO AMBITO DA INTERNET)': 'AMBIENTE VIRTUAL',
    'AMBIENTE VIRTUAL (INTERNET/REDE SOCIAL/APLICATIVOS)': 'AMBIENTE VIRTUAL',

    'LOCAL DE TRABALHO DA VITIMA': 'LOCAL DE TRABALHO DA VITIMA/SUSPEITO',
    'LOCAL DE TRABALHO DA VITMA': 'LOCAL DE TRABALHO DA VITIMA/SUSPEITO',
    'LOCAL DE TRABALHO DO AGRESSOR': 'LOCAL DE TRABALHO DA VITIMA/SUSPEITO',

    'ORGAO PUBLICO': 'ORGAO PUBLICO',
    'ORGAOS PUBLICOS': 'ORGAO PUBLICO',
    'DEFENSORIA PUBLICA': 'ORGAO PUBLICO',
    'JUIZADO': 'ORGAO PUBLICO',

    'DELEGACIA': 'DELEGACIA DE POLICIA',
    'DELEGACIA DE POLICIA': 'DELEGACIA DE POLICIA',
    'DELEGACIA DE POLICIA COMO UNIDADE PRISIONAL': 'DELEGACIA DE POLICIA',

    'UNIDADE PRISIONAL': 'UNIDADE PRISIONAL',

    'ESTABELECIMENTO DE SAUDE': 'HOSPITAL, MANICOMIO, ESTABELECIMENTO DE SAUDE',
    'HOSPITAL': 'HOSPITAL, MANICOMIO, ESTABELECIMENTO DE SAUDE',
    'MANICOMIO/HOSPITAL PSIQUIATRICO/CASA DE SAUDE MENTAL': 'HOSPITAL, MANICOMIO, ESTABELECIMENTO DE SAUDE',

    'INSTITUICAO DE ENSINO': 'INSTITUICAO/ESTABELECIMENTO DE ENSINO',
    'ESTABELECIMENTO DE ENSINO': 'INSTITUICAO/ESTABELECIMENTO DE ENSINO',
    'ESCOLA': 'INSTITUICAO/ESTABELECIMENTO DE ENSINO',
    'BERCARIO/CRECHE': 'INSTITUICAO/ESTABELECIMENTO DE ENSINO',
    'CENTRO DE TREINAMENTO': 'INSTITUICAO/ESTABELECIMENTO DE ENSINO',


    'AMBIENTE DE LAZER': 'AMBIENTE DE LAZER (SHOPPING, CINEMA, BAR, LANCHONETE, CASA NOTURNA, CLUBE ESPORTIVO, ETC)',
    'AMBIENTE DE LAZER/ESPORTE/ENTRETENIMENTO': 'AMBIENTE DE LAZER (SHOPPING, CINEMA, BAR, LANCHONETE, CASA NOTURNA, CLUBE ESPORTIVO, ETC)',
    'SHOPPING, CINEMA, CLUBE OU OUTRO AMBIENTE DE LAZER': 'AMBIENTE DE LAZER (SHOPPING, CINEMA, BAR, LANCHONETE, CASA NOTURNA, CLUBE ESPORTIVO, ETC)',
    'SHOPPING': 'AMBIENTE DE LAZER (SHOPPING, CINEMA, BAR, LANCHONETE, CASA NOTURNA, CLUBE ESPORTIVO, ETC)',
    'BAR, LANCHONETE OU CASA NOTURNA': 'AMBIENTE DE LAZER (SHOPPING, CINEMA, BAR, LANCHONETE, CASA NOTURNA, CLUBE ESPORTIVO, ETC)',
    'BAR, LANCHONETE, RESTAURANTE': 'AMBIENTE DE LAZER (SHOPPING, CINEMA, BAR, LANCHONETE, CASA NOTURNA, CLUBE ESPORTIVO, ETC)',
    'BAR, RESTAURANTE, LANCHONETE': 'AMBIENTE DE LAZER (SHOPPING, CINEMA, BAR, LANCHONETE, CASA NOTURNA, CLUBE ESPORTIVO, ETC)',
    'CASA NOTURNA': 'AMBIENTE DE LAZER (SHOPPING, CINEMA, BAR, LANCHONETE, CASA NOTURNA, CLUBE ESPORTIVO, ETC)',
    'CLUBE ESPORTIVO': 'AMBIENTE DE LAZER (SHOPPING, CINEMA, BAR, LANCHONETE, CASA NOTURNA, CLUBE ESPORTIVO, ETC)',
    'CINEMA': 'AMBIENTE DE LAZER (SHOPPING, CINEMA, BAR, LANCHONETE, CASA NOTURNA, CLUBE ESPORTIVO, ETC)',

    'ONIBUS': 'TRANSPORTE (ONIBUS, METRO, TREM, TAXI, TRANSPORTE MONITORADO POR APLICATIVO, TRANSPORTE MARITIMO, TRANSPORTE AEREO)',
    'METRO/TREM': 'TRANSPORTE (ONIBUS, METRO, TREM, TAXI, TRANSPORTE MONITORADO POR APLICATIVO, TRANSPORTE MARITIMO, TRANSPORTE AEREO)',
    'METRO OU TREM': 'TRANSPORTE (ONIBUS, METRO, TREM, TAXI, TRANSPORTE MONITORADO POR APLICATIVO, TRANSPORTE MARITIMO, TRANSPORTE AEREO)',
    'TAXI/TRANSPORTE APLICATIVO': 'TRANSPORTE (ONIBUS, METRO, TREM, TAXI, TRANSPORTE MONITORADO POR APLICATIVO, TRANSPORTE MARITIMO, TRANSPORTE AEREO)',
    'TAXI': 'TRANSPORTE (ONIBUS, METRO, TREM, TAXI, TRANSPORTE MONITORADO POR APLICATIVO, TRANSPORTE MARITIMO, TRANSPORTE AEREO)',
    'TRANSPORTE MONITORADO POR APLICATIVO': 'TRANSPORTE (ONIBUS, METRO, TREM, TAXI, TRANSPORTE MONITORADO POR APLICATIVO, TRANSPORTE MARITIMO, TRANSPORTE AEREO)',
    'TRANSPORTE MARITIMO': 'TRANSPORTE (ONIBUS, METRO, TREM, TAXI, TRANSPORTE MONITORADO POR APLICATIVO, TRANSPORTE MARITIMO, TRANSPORTE AEREO)',
    'TRANSPORTE AEREO': 'TRANSPORTE (ONIBUS, METRO, TREM, TAXI, TRANSPORTE MONITORADO POR APLICATIVO, TRANSPORTE MARITIMO, TRANSPORTE AEREO)',

    'CENTRO DE REFERENCIA': 'SERVICOS SOCIAIS (CENTRO DE REFERENCIA, SERVICO DE ABRIGAMENTO, UNIDADE DE MEDIDA SOCIOEDUCATIVA, INSTITUICAO DE LONGA PERMANENCIA DE IDOSO - ILPI)',
    'SERVICO DE ABRIGAMENTO': 'SERVICOS SOCIAIS (CENTRO DE REFERENCIA, SERVICO DE ABRIGAMENTO, UNIDADE DE MEDIDA SOCIOEDUCATIVA, INSTITUICAO DE LONGA PERMANENCIA DE IDOSO - ILPI)',
    'UNIDADE DE MEDIDA SOCIOEDUCATIVA': 'SERVICOS SOCIAIS (CENTRO DE REFERENCIA, SERVICO DE ABRIGAMENTO, UNIDADE DE MEDIDA SOCIOEDUCATIVA, INSTITUICAO DE LONGA PERMANENCIA DE IDOSO - ILPI)',
    'INSTITUICAO DE LONGA PERMANENCIA DE IDOSO - ILPI': 'SERVICOS SOCIAIS (CENTRO DE REFERENCIA, SERVICO DE ABRIGAMENTO, UNIDADE DE MEDIDA SOCIOEDUCATIVA, INSTITUICAO DE LONGA PERMANENCIA DE IDOSO - ILPI)',
    'INSTITUICAO DE LONGA PERMANENCIA PARA IDOSOS - ILPI': 'SERVICOS SOCIAIS (CENTRO DE REFERENCIA, SERVICO DE ABRIGAMENTO, UNIDADE DE MEDIDA SOCIOEDUCATIVA, INSTITUICAO DE LONGA PERMANENCIA DE IDOSO - ILPI)',

    'EVENTO PUBLICO': 'EVENTO PUBLICO',

    'ESTABELECIMENTO COMERCIAL': 'ESTABELECIMENTO COMERCIAL',
    'LOJA': 'ESTABELECIMENTO COMERCIAL',

    'INSTITUICAO FINANCEIRA': 'INSTITUICAO FINANCEIRA',

    'OUTROS': 'OUTROS',
}
novaTabelaDeTodoPeriodo['Cenário da violação'] = novaTabelaDeTodoPeriodo['Cenário da violação'].replace(substituicoes)

substituicoes = {
    'COMPANHEIRO(A)': 'PARCEIRO(A) ATUAL',
    'ESPOSA(O)': 'PARCEIRO(A) ATUAL',
    'CONJUGE': 'PARCEIRO(A) ATUAL',
    'NAMORADO(A)': 'PARCEIRO(A) ATUAL',
    'MARIDO/ESPOSA': 'PARCEIRO(A) ATUAL',
    'ESPOSO(A)': 'PARCEIRO(A) ATUAL',
    'NAMORADO(A)/COMPANHEIRO(A)': 'PARCEIRO(A) ATUAL',

    'EX-COMPANHEIRO(A)': 'EX-PARCEIRO(A)',
    'EX-ESPOSA(O)': 'EX-PARCEIRO(A)',
    'EX-ESPOSO(A)': 'EX-PARCEIRO(A)',
    'EX-NAMORADO(A)': 'EX-PARCEIRO(A)',
    'EX-NAMORADO(A)/EX-COMPANHEIRO(A)': 'EX-PARCEIRO(A)',
    'EX-CONJUGE': 'EX-PARCEIRO(A)',
    'EX-MARIDO': 'EX-PARCEIRO(A)',
    'AMANTE': 'EX-PARCEIRO(A)',

    'MAE': 'PAI/MAE',
    'PAI': 'PAI/MAE',
    'PAI/MAE': 'PAI/MAE',

    'FILHO(A)': 'FILHO(A)/ENTEADO(A)',
    'FILHO(A)/ENTEADO(A)': 'FILHO(A)/ENTEADO(A)',
    'ENTEADO(A)': 'FILHO(A)/ENTEADO(A)',

    'IRMAO(A)': 'IRMAO(A)',
    'TIO(A)': 'TIO(A)',
    'SOBRINHO(A)': 'SOBRINHO(A)',
    'PRIMO(A)': 'PRIMO(A)',

    'AVO': 'AVO(O)/BISAVO(O)/TRISAVO(O)',
    'AVO(O)': 'AVO(O)/BISAVO(O)/TRISAVO(O)',
    'BISAVO(O)': 'AVO(O)/BISAVO(O)/TRISAVO(O)',
    'TRISAVO(O)': 'AVO(O)/BISAVO(O)/TRISAVO(O)',

    'NETO(A)': 'NETO(A)/BISNETO(A)',
    'BISNETO(A)': 'NETO(A)/BISNETO(A)',

    'GENRO/NORA': 'GENRO/NORA',

    'SOGRO(A)': 'SOGRO(A)',

    'CUNHADO(A)': 'CUNHADO(A)',

    'PADRASTO/MADRASTA': 'PADRASTO/MADRASTA',
    'PADRASTO': 'PADRASTO/MADRASTA',
    'MADRASTA': 'PADRASTO/MADRASTA',

    'PADRINHO/MADRINHA': 'PADRINHO/MADRINHA',
    'COMPANHEIRO(A) DA MAE/DO PAI': 'COMPANHEIRO(A) DA MAE/DO PAI',

    'FAMILIARES': 'OUTROS FAMILIAR',
    'PARENTES': 'OUTROS FAMILIAR',

    'AMIGO(A)': 'AMIGO/CONHECIDO',
    'AMIGO(A) DA FAMILIA': 'AMIGO/CONHECIDO',
    'CONHECIDO(A)': 'AMIGO/CONHECIDO',
    'VIZINHO(A)': 'AMIGO/CONHECIDO',
    'MORA/MOROU NA MESMA RESIDENCIA MAS NAO E FAMILIAR': 'AMIGO/CONHECIDO',
    'PROPRIETARIO(A) DO IMOVEL EM QUE A VITIMA RESIDE': 'AMIGO/CONHECIDO',

    'COLEGA DE TRABALHO': 'COLEGA DE TRABALHO',
    'COLEGA DE TRABALHO (MESMO NIVEL HIERARQUICO)': 'COLEGA DE TRABALHO',

    'EMPREGADOR': 'EMPREGADOR/PATRAO (HIERARQUICAMENTE SUPERIOR)',
    'EMPREGADOR/PATRAO (HIERARQUICAMENTE SUPERIOR)': 'EMPREGADOR/PATRAO (HIERARQUICAMENTE SUPERIOR)',
    
    'EMPREGADO(A)': 'EMPREGADO(A)',
    'EMPREGADO(A) DOMESTICO': 'EMPREGADO(A)',
    'SUBORDINADO': 'EMPREGADO(A)',
    'FUNCIONARIO(A)': 'EMPREGADO(A)',
    'FUNCIONARIO, VOLUNTARIO OU PRESTADOR DE SERVICO PARA INSTITUICAO': 'EMPREGADO(A)',

    'CHEFE': 'CHEFE(A)',
    'CHEFE(A)': 'CHEFE(A)',

    'HIERARQUICAMENTE INFERIOR': 'HIERARQUICAMENTE INFERIOR',
    'HIERARQUICAMENTE SUPERIOR': 'HIERARQUICAMENTE SUPERIOR',
    'MESMO NIVEL HIERARQUICO': 'MESMO NIVEL HIERARQUICO',

    'PROFISSIONAL DE SAUDE': 'PROFISSIONAL DE SAUDE',
    'PROFISSIONAL DA SAUDE': 'PROFISSIONAL DE SAUDE',

    'PROFISSIONAL DA SEGURANCA PUBLICA': 'PROFISSIONAL DA SEGURANCA PUBLICA',

    'CUIDADOR(A)': 'PROFISSIONAL',
    'CUIDADOR (A)': 'PROFISSIONAL',

    'PROFESSOR(A)': 'PROFESSOR(A)',
    
    'DIRETOR(A) DE ESCOLA': 'DIRETOR(A)/GESTOR(A) DE INSTITUICAO',
    'DIRETOR(A)': 'DIRETOR(A)/GESTOR(A) DE INSTITUICAO',
    'DIRETOR(A) DE UNIDADE PRISIONAL': 'DIRETOR(A)/GESTOR(A) DE INSTITUICAO',
    'DIRETOR(A) DA UNIDADE PRISIONAL': 'DIRETOR(A)/GESTOR(A) DE INSTITUICAO',
    'DIRETOR/GESTOR DE INSTITUICAO': 'DIRETOR(A)/GESTOR(A) DE INSTITUICAO',

    'TREINADOR(A)/TECNICO(A)': 'TREINADOR(A)/TECNICO(A)',
    'LIDER RELIGIOSO(A)': 'LIDER RELIGIOSO(A)',
    'AGENTE PENITENCIARIO': 'AGENTE PENITENCIARIO',
    'ALUNO(A)': 'ALUNO(A)',
    'INTERNO(A)': 'INTERNO(A)',
    'PRESTADOR(A) DE SERVICO': 'PRESTADOR(A) DE SERVICO',

    'DESCONHECIDO(A)': 'DESCONHECIDO',
    'SEM RELACAO ': 'DESCONHECIDO',
    'NAO SE APLICA': 'DESCONHECIDO',

    'OUTROS': 'OUTROS',
    'OUTRO(A)': 'OUTROS',
    'PROPRIA VITIMA': 'OUTROS',
    'PESSOA COM QUEM MANTEM/MANTEVE CONVIVENCIA FAMILIAR': 'OUTROS',
    'MORA/MOROU NA MESMA RESIDENCIA MAS NAO E FAMILIAR': 'OUTROS',
}
novaTabelaDeTodoPeriodo['Relação vítima-suspeito'] = novaTabelaDeTodoPeriodo['Relação vítima-suspeito'].replace(substituicoes)

In [40]:
novaTabelaDeTodoPeriodo.to_csv(nomeDoNovoArquivoDeTodoPeriodo, index=False)

# 7) Saves the updated field comparison file

In [19]:
tabelaComNomesDasColunas.to_excel("Comparação dos campos de cada tabela.xlsx", index=False)